In [65]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PowerTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import mutual_info_classif, SelectKBest
import optuna
import shap

In [66]:
class SMOTETransformer(BaseEstimator, TransformerMixin):
    def __init__(self, random_state=42):
        self.random_state = random_state
        self.smote = SMOTE(random_state=self.random_state)

    def fit_resample(self, X, y):
        return self.smote.fit_resample(X, y)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X

In [40]:
# Load the data
train_data = pd.read_csv('input/train.csv')
test_data = pd.read_csv('input/test.csv')

# Impute missing ages with median
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].median())

# Impute missing embarkments
most_frequent_embarked = train_data['Embarked'].mode()[0]
train_data['Embarked'] = train_data['Embarked'].fillna(most_frequent_embarked)
test_data['Embarked'] = test_data['Embarked'].fillna(most_frequent_embarked)

# Combine train and test for preprocessing
all_data = pd.concat([train_data, test_data], sort=False).reset_index(drop=True)

In [41]:
# Feature Engineering

# Extract titles from names
all_data['Title'] = all_data['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
all_data['Title'] = all_data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
all_data['Title'] = all_data['Title'].replace('Mlle', 'Miss')
all_data['Title'] = all_data['Title'].replace('Ms', 'Miss')
all_data['Title'] = all_data['Title'].replace('Mme', 'Mrs')

# Create family size feature
all_data['FamilySize'] = all_data['SibSp'] + all_data['Parch'] + 1

# Create is_alone feature
all_data['IsAlone'] = 0
all_data.loc[all_data['FamilySize'] == 1, 'IsAlone'] = 1

# Extract deck from cabin
all_data['Deck'] = all_data['Cabin'].str.slice(0,1)
all_data['Deck'] = all_data['Deck'].fillna('U')

# Bin age
all_data['AgeBin'] = pd.cut(all_data['Age'], bins=[0, 12, 20, 40, 60, 100], labels=['Child', 'Teen', 'Adult', 'Senior', 'Elderly'])

# Bin fare
all_data['FareBin'] = pd.qcut(all_data['Fare'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

# Create interaction features
all_data['Age*Class'] = all_data['Age'] * all_data['Pclass']
all_data['Fare*Class'] = all_data['Fare'] * all_data['Pclass']

In [73]:
# Feature Selection
numeric_features = ['Age', 'Fare', 'FamilySize']
categorical_features = ['Pclass', 'Sex', 'Embarked', 'Title', 'Deck', 'AgeBin', 'FareBin', 'IsAlone']

# Split back into train and test
train_data = all_data[all_data['Survived'].notna()].copy()
test_data = all_data[all_data['Survived'].isna()].copy()

X = train_data[numeric_features + categorical_features]
y = train_data['Survived']

X.head(5)

,Age,Fare,FamilySize,Pclass,Sex,Embarked,Title,Deck,AgeBin,FareBin,IsAlone
0,22.0,7.2500,2,3,male,S,Mr,U,Adult,Low,0
1,38.0,71.2833,2,1,female,C,Mrs,C,Adult,Very High,0
2,26.0,7.9250,1,3,female,S,Miss,U,Adult,Medium,1
3,35.0,53.1000,2,1,female,S,Mrs,C,Adult,Very High,0
4,35.0,8.0500,1,3,male,S,Mr,U,Adult,Medium,1


In [77]:
# Encode categorical variables
categorical_encoder = OneHotEncoder(drop='first', sparse_output=False)
X_encoded = pd.DataFrame(categorical_encoder.fit_transform(X[categorical_features]))
X_encoded.columns = categorical_encoder.get_feature_names_out(categorical_features)

# Combine with numeric features
X_numeric = X[numeric_features].reset_index(drop=True)
X_preprocessed = pd.concat([X_numeric, X_encoded], axis=1)

In [78]:
# Mutual Information for feature selection
mi_scores = mutual_info_classif(X_encoded, y)
mi_scores = pd.Series(mi_scores, index=X_encoded.columns)
mi_scores = mi_scores.sort_values(ascending=False)
top_features = mi_scores.nlargest(10).index.tolist()

print("\nTop 10 features based on mutual information:")
print(mi_scores.nlargest(10))


Top 10 features based on mutual information:
Sex_male             0.162521
Title_Mr             0.152560
Title_Mrs            0.083310
Title_Miss           0.062254
Pclass_3             0.035179
FareBin_Very High    0.022639
AgeBin_Child         0.019521
Deck_C               0.018392
Deck_U               0.016980
Embarked_S           0.016954
dtype: float64


In [79]:
# Advanced Preprocessing
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', PowerTransformer())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [80]:
# Define the base models
base_models = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('xgb', XGBClassifier(random_state=42)),
    ('lgbm', LGBMClassifier(random_state=42))
]

# Define the meta-model
meta_model = LogisticRegression()

# Create the stacking ensemble
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

In [ ]:
# Hyperparameter Optimization with Optuna
def objective(trial):
    rf_n_estimators = trial.suggest_int('rf_n_estimators', 100, 1000)
    rf_max_depth = trial.suggest_int('rf_max_depth', 2, 32)
    xgb_n_estimators = trial.suggest_int('xgb_n_estimators', 100, 1000)
    xgb_max_depth = trial.suggest_int('xgb_max_depth', 2, 32)
    lgbm_n_estimators = trial.suggest_int('lgbm_n_estimators', 100, 1000)
    lgbm_max_depth = trial.suggest_int('lgbm_max_depth', 2, 32)

    lgbm_params = {
    'n_estimators': lgbm_n_estimators,
    'max_depth': lgbm_max_depth,
    'min_child_samples': trial.suggest_int('lgbm_min_child_samples', 1, 50),
    'min_gain_to_split': trial.suggest_float('lgbm_min_gain_to_split', 0, 1),
    'random_state': 42
    }
    
    base_models = [
        ('rf', RandomForestClassifier(n_estimators=rf_n_estimators, max_depth=rf_max_depth, random_state=42)),
        ('xgb', XGBClassifier(n_estimators=xgb_n_estimators, max_depth=xgb_max_depth, random_state=42)),
        ('lgbm', LGBMClassifier(**lgbm_params))
    ]
    
    stacking_model = StackingClassifier(
        estimators=base_models,
        final_estimator=LogisticRegression(),
        cv=5
    )
    
    pipeline = Pipeline([
        ('feature_selection', SelectKBest(score_func=mutual_info_classif, k=10)),
        ('classifier', stacking_model)
    ])

    smote = SMOTE(random_state=42)
    
    def custom_cross_val_score(estimator, X, y, cv):
        skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
        scores = []
        
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
            
            estimator.fit(X_train_resampled, y_train_resampled)
            scores.append(estimator.score(X_test, y_test))
        
        return np.array(scores)
    
    score = np.mean(custom_cross_val_score(pipeline, X_preprocessed, y, cv=5))
    return score

In [ ]:
# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get best parameters
best_params = study.best_params
print("Best parameters:", best_params)
print("Best score:", study.best_value)